In [2]:
# build_db.py
import duckdb, pandas as pd, zipfile, io
from pyproj import Transformer

GTFS_ZIP = "gtfs_m.zip"  
DB_PATH  = "mta_gtfs.duckdb"

## Load and setup data

### Load GTFS files

In [10]:
def read_csv(z, name, **kw):
    with z.open(name) as f:
        return pd.read_csv(io.TextIOWrapper(f, encoding=kw.pop("encoding", "utf-8")), **kw)

def to_sec(hms):
    # supports times like 24:15:00+
    h, m, s = map(int, hms.split(":"))
    return h*3600 + m*60 + s

with zipfile.ZipFile(GTFS_ZIP) as z:
    routes = read_csv(z, "routes.txt")
    trips  = read_csv(z, "trips.txt")
    stops  = read_csv(z, "stops.txt")
    stimes = read_csv(z, "stop_times.txt")
    cal    = read_csv(z, "calendar.txt")
    caldates = read_csv(z, "calendar_dates.txt") if "calendar_dates.txt" in z.namelist() else pd.DataFrame()

# times → seconds
stimes["arrival_sec"] = stimes["arrival_time"].map(to_sec)
# base stop level only
# stops = stops.rename(columns={"stop_lat":"lat","stop_lon":"lon"})
stops = stops.loc[stops.get("location_type", 0).fillna(0).eq(0), ["stop_id","stop_name","stop_lat","stop_lon","parent_station"]]

# project to EPSG:2263 (feet-based CRS for NYC)
tf = Transformer.from_crs("EPSG:4326", "EPSG:2263", always_xy=True)
x, y = tf.transform(stops["stop_lon"].values, stops["stop_lat"].values)
stops["x2263"] = x
stops["y2263"] = y

### Create Database

In [15]:
con = duckdb.connect(DB_PATH)
con.execute("INSTALL spatial; LOAD spatial;")  # lets us use ST_* if desired

# write tables
con.register("routes_df", routes)
con.register("trips_df", trips)
con.register("stops_df", stops)
con.register("stimes_df", stimes)
con.register("cal_df", cal)
if not caldates.empty:
    con.register("caldates_df", caldates)


# Base/source tables (persisted) -> raw tables from GTFS data
con.execute("CREATE OR REPLACE TABLE routes_base AS SELECT * FROM routes_df")
con.execute("""
            CREATE OR REPLACE TABLE trips_base AS
            SELECT trip_id, route_id, direction_id, trip_headsign, service_id
            FROM trips_df
""")
con.execute("""
            CREATE OR REPLACE TABLE stop_times_base AS
            SELECT trip_id, stop_id, stop_sequence, arrival_sec  -- arrival_sec already added in pandas
            FROM stimes_df
""")
con.execute("""
            CREATE OR REPLACE TABLE calendar_base AS
            SELECT CAST(start_date AS VARCHAR) AS start_date,
                CAST(end_date   AS VARCHAR) AS end_date,
                service_id, monday,tuesday,wednesday,thursday,friday,saturday,sunday
            FROM cal_df
""")
if not caldates.empty:
    con.execute("""
    CREATE OR REPLACE TABLE calendar_dates_base AS
    SELECT service_id, CAST(date AS VARCHAR) AS date, exception_type
    FROM caldates_df
    """)

# Dim tables (persisted) -> reference tables (cleaned/descriptive lookup tables)
con.execute("CREATE OR REPLACE TABLE dim_routes AS SELECT * FROM routes_base")
con.execute("""
CREATE OR REPLACE TABLE dim_stops  AS
SELECT stop_id, stop_name, stop_lat, stop_lon, parent_station, x2263, y2263
FROM stops_df
""")

# Fact table (persisted) -> big event table, every stop arrival (to be queried from dim tables)
con.execute("""
CREATE OR REPLACE TABLE fact_stop_events AS
SELECT
  t.route_id,
  t.direction_id,
  t.service_id,
  st.stop_id,
  st.stop_sequence,
  st.arrival_sec,
  t.trip_id
FROM stop_times_base st
JOIN trips_base      t ON st.trip_id = t.trip_id
""")

# Convenience copy of trips with just what you need (persisted)
con.execute("""
CREATE OR REPLACE TABLE dim_trips AS
SELECT trip_id, route_id, direction_id, trip_headsign, service_id
FROM trips_base
""")

# helper VIEWS for day types
con.execute("""
CREATE OR REPLACE VIEW svcs_weekday AS
SELECT DISTINCT service_id
FROM calendar_base
WHERE monday=1 OR tuesday=1 OR wednesday=1 OR thursday=1 OR friday=1
""")
con.execute("""
CREATE OR REPLACE VIEW svcs_saturday AS
SELECT DISTINCT service_id
FROM calendar_base
WHERE saturday=1
""")
con.execute("""
CREATE OR REPLACE VIEW svcs_sunday AS
SELECT DISTINCT service_id
FROM calendar_base
WHERE sunday=1
""")


print(f"✅ Built {DB_PATH}")

✅ Built mta_gtfs.duckdb


## Explore DuckDB

In [ ]:
# con.close()

In [22]:
con = duckdb.connect(DB_PATH, read_only=True)

# If you want spatial functions (ST_*):
con.execute("INSTALL spatial; LOAD spatial;")

In [23]:
# List tables
con.execute("SHOW ALL TABLES").fetchdf()

,database,schema,name,column_names,column_types,temporary
0,mta_gtfs,main,calendar,"[start_date, end_date, service_id, monday, tue...","[VARCHAR, VARCHAR, VARCHAR, BIGINT, BIGINT, BI...",False
1,mta_gtfs,main,calendar_base,"[start_date, end_date, service_id, monday, tue...","[VARCHAR, VARCHAR, VARCHAR, BIGINT, BIGINT, BI...",False
2,mta_gtfs,main,calendar_dates,"[service_id, date, exception_type]","[VARCHAR, VARCHAR, BIGINT]",False
3,mta_gtfs,main,calendar_dates_base,"[service_id, date, exception_type]","[VARCHAR, VARCHAR, BIGINT]",False
4,mta_gtfs,main,dim_routes,"[route_id, agency_id, route_short_name, route_...","[VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, ...",False
5,mta_gtfs,main,dim_stops,"[stop_id, stop_name, stop_lat, stop_lon, paren...","[BIGINT, VARCHAR, DOUBLE, DOUBLE, DOUBLE, DOUB...",False
6,mta_gtfs,main,dim_trips,"[trip_id, route_id, direction_id, trip_headsig...","[VARCHAR, VARCHAR, BIGINT, VARCHAR, VARCHAR]",False
7,mta_gtfs,main,fact_stop_events,"[route_id, direction_id, service_id, stop_id, ...","[VARCHAR, BIGINT, VARCHAR, BIGINT, BIGINT, BIG...",False
8,mta_gtfs,main,routes_base,"[route_id, agency_id, route_short_name, route_...","[VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, ...",False
9,mta_gtfs,main,stop_times_base,"[trip_id, stop_id, stop_sequence, arrival_sec]","[VARCHAR, BIGINT, BIGINT, BIGINT]",False


In [17]:
# Peek at a table
con.execute("SELECT * FROM dim_stops LIMIT 5").fetchdf()

,stop_id,stop_name,stop_lat,stop_lon,parent_station,x2263,y2263
0,101014,WILLIS AV/E 138 ST,40.808930,-73.922855,NaN,1.005606e+06,234001.482416
1,101015,WILLIS AV/E 140 ST,40.810634,-73.921632,NaN,1.005944e+06,234622.610449
2,101017,WILLIS AV/E 144 ST,40.813111,-73.919823,NaN,1.006444e+06,235525.522682
3,101018,WILLIS AV/E 146 ST,40.814404,-73.918954,NaN,1.006684e+06,235996.829999
4,101066,3 AV/E 150 ST,40.816142,-73.917547,NaN,1.007073e+06,236630.408913


In [18]:
con.execute("SELECT * FROM dim_routes LIMIT 5").fetchdf()

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_color,route_text_color
0,B1,MTA NYCT,B1,Bay Ridge - Manhattan Beach,via 86th St / Ocean Pkwy,3,00AEEF,FFFFFF
1,B11,MTA NYCT,B11,Sunset Park - Midwood,via 49th & 50th St / Avenue J,3,006CB7,FFFFFF
2,B12,MTA NYCT,B12,Lefferts Gardens - East New York,via Clarkson Av / Empire Blvd / East New York Av,3,6CBE45,FFFFFF
3,B13,MTA NYCT,B13,Spring Creek - Wyckoff Hospital,via Crescent St / Jamaica Av / Wyckoff Av,3,FAA61A,FFFFFF
4,B14,MTA NYCT,B14,Spring Creek - Crown Heights,via Sutter Av / Pitkin Av,3,00AEEF,FFFFFF


In [19]:
con.execute("SELECT * FROM calendar LIMIT 5").fetchdf()

,start_date,end_date,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday
0,20250831,20260101,MQ_D5-Sunday,0,0,0,0,0,0,1
1,20250902,20260102,MQ_D5-Weekday-SDon,1,1,1,1,1,0,0
2,20250906,20260103,MQ_D5-Saturday,0,0,0,0,0,1,0
3,20251013,20251231,MQ_O5-Weekday,1,1,1,1,1,0,0
4,20250831,20260101,MV_D5-Sunday,0,0,0,0,0,0,1


In [ ]:
# schema
con.execute("DESCRIBE dim_stops").fetchdf()         # or:
con.execute("PRAGMA table_info('dim_stops')").fetchdf()

In [20]:
con.execute("""
SELECT table_schema, table_name, column_name, data_type
FROM information_schema.columns
WHERE table_schema='main'
ORDER BY table_name, ordinal_position
""").fetchdf()

,table_schema,table_name,column_name,data_type
0,main,cal_df,service_id,VARCHAR
1,main,cal_df,monday,BIGINT
2,main,cal_df,tuesday,BIGINT
3,main,cal_df,wednesday,BIGINT
4,main,cal_df,thursday,BIGINT
...,...,...,...,...
112,main,trips_df,trip_id,VARCHAR
113,main,trips_df,trip_headsign,VARCHAR
114,main,trips_df,direction_id,BIGINT
115,main,trips_df,block_id,BIGINT


### Query Testing

In [27]:
stop_id = "402705"  # your stop_id
start_sec, end_sec = 7*3600+45*60, 8*3600+45*60  # 07:45–08:45

df = con.execute("""
WITH win AS (SELECT ? AS s, ? AS e)
SELECT
  r.route_id,
  t.direction_id,
  COUNT(*) AS buses_scheduled
FROM fact_stop_events f
JOIN dim_trips t  ON f.trip_id = t.trip_id
JOIN dim_routes r ON r.route_id = t.route_id
JOIN svcs_weekday v ON v.service_id = f.service_id   -- pick svcs_saturday / svcs_sunday as needed
CROSS JOIN win
WHERE f.stop_id = ?
  AND f.arrival_sec BETWEEN (SELECT s FROM win) AND (SELECT e FROM win)
GROUP BY r.route_id, t.direction_id
ORDER BY r.route_id, t.direction_id
""", [start_sec, end_sec, stop_id]).fetchdf()

df

,route_id,direction_id,buses_scheduled
0,M101,0,1
1,M102,0,8
2,M103,0,6


In [30]:
# Example intersection (lon, lat)
lon, lat = -73.998522, 40.745306

from pyproj import Transformer
tf = Transformer.from_crs("EPSG:4326", "EPSG:2263", always_xy=True)
x0, y0 = tf.transform(lon, lat)

nearby = con.execute("""
SELECT stop_id, stop_name, stop_lat, stop_lon
FROM dim_stops
WHERE (x2263 - ?)*(x2263 - ?) + (y2263 - ?)*(y2263 - ?) <= ?*?
""", [x0, x0, y0, y0, 250, 250]).fetchdf()
nearby

,stop_id,stop_name,stop_lat,stop_lon
0,402126,W 23 ST/8 AV,40.745059,-73.998141
1,402154,W 23 ST/8 AV,40.745547,-73.998888
2,405623,8 AV/W 23 ST,40.744919,-73.998656


if 250 is different intersection
add intersection, block?
polygon points?

near side/far side, block

peak hours
number of stops for each route and each direction
for each stop in a polygon/area

multiple intersections?
intersection name


In [37]:
def buses_by_stop_route_dir_within_radius(
    lon: float,
    lat: float,
    start_time: str,    # e.g., "07:45"
    end_time: str,      # e.g., "08:45"
    day_type: str,      # "Weekday" | "Saturday" | "Sunday"
    radius_ft: int = 250,
) -> pd.DataFrame:
    
    def to_sec(hms: str) -> int:
      hh, mm, *rest = hms.split(":")
      ss = int(rest[0]) if rest else 0
      return int(hh)*3600 + int(mm)*60 + ss

    # project input point to EPSG:2263 (feet) to measure feet distances
    tf = Transformer.from_crs("EPSG:4326", "EPSG:2263", always_xy=True)
    x0, y0 = tf.transform(lon, lat)
    start_sec, end_sec = to_sec(start_time), to_sec(end_time)

    sql = """
    WITH svcs AS (
      SELECT DISTINCT service_id
      FROM calendar_base
      WHERE
        (? = 'Weekday'  AND (monday=1 OR tuesday=1 OR wednesday=1 OR thursday=1 OR friday=1))
        OR (? = 'Saturday' AND saturday=1)
        OR (? = 'Sunday'   AND sunday=1)
    ),
    win AS (
      SELECT ?::INTEGER AS s, ?::INTEGER AS e
    ),
    near_stops AS (
      SELECT stop_id, stop_name, stop_lat, stop_lon
      FROM dim_stops
      WHERE (x2263 - ?)*(x2263 - ?) + (y2263 - ?)*(y2263 - ?) <= ?*?
    )
    SELECT
      r.route_id,
      t.direction_id,
      s.stop_id,
      s.stop_name,
      s.stop_lat AS stop_lat,
      s.stop_lon AS stop_lon,
      COUNT(*) AS buses_scheduled
    FROM fact_stop_events f
    JOIN dim_trips  t ON f.trip_id = t.trip_id
    JOIN dim_routes r ON t.route_id = r.route_id
    JOIN svcs       v ON v.service_id = f.service_id
    JOIN near_stops s ON s.stop_id   = f.stop_id
    CROSS JOIN win
    WHERE f.arrival_sec BETWEEN (SELECT s FROM win) AND (SELECT e FROM win)  -- inclusive
    GROUP BY r.route_id, t.direction_id, s.stop_id, s.stop_name, s.stop_lat, s.stop_lon
    ORDER BY s.stop_name, r.route_id, t.direction_id;
    """

    # note: day_type is passed 3 times for the svcs CTE; that's intentional
    params = [day_type, day_type, day_type,
              start_sec, end_sec,
              x0, x0, y0, y0, radius_ft, radius_ft]

    return con.execute(sql, params).fetchdf()

In [38]:
lon, lat = -73.998522, 40.745306            # your intersection
df = buses_by_stop_route_dir_within_radius(
    lon=lon, lat=lat,
    start_time="07:45", end_time="08:45",
    day_type="Weekday",                      # or "Saturday"/"Sunday"
    radius_ft=250
)
df

,route_id,direction_id,stop_id,stop_name,stop_lat,stop_lon,buses_scheduled
0,M20,0,405623,8 AV/W 23 ST,40.744919,-73.998656,6
1,M23+,0,402126,W 23 ST/8 AV,40.745059,-73.998141,15
2,M23+,1,402154,W 23 ST/8 AV,40.745547,-73.998888,14
